In [1]:
import ee
import os
import geopandas as gpd
import glob
import eeconvert
import numpy as np
import cv2
import rasterio
from pyproj import Transformer


The final output file (e.g., image.tif) will contain two bands:

Band 1: Slope

Band 2: Relative Elevation

In [2]:
ee.Initialize(project='55961222436')


In [3]:
import os
import rasterio
import ee
import numpy as np
import requests
from osgeo import gdal
from pyproj import Transformer
import subprocess

# Initialize Google Earth Engine
ee.Initialize()

# Function to extract EPSG and bounding box from GeoTIFF
# had to add a small buffer to encapsulate the entire image
def get_image_metadata(tif_path, buffer_size=0.05):
    with rasterio.open(tif_path) as dataset:
        epsg_code = dataset.crs.to_epsg()
        bounds = dataset.bounds  # (left, bottom, right, top)

        # Transform bounding box to EPSG:4326
        transformer = Transformer.from_crs(epsg_code, 4326, always_xy=True)
        xmin, ymin = transformer.transform(bounds.left, bounds.bottom)
        xmax, ymax = transformer.transform(bounds.right, bounds.top)

        # Add buffer to the region's bounding box
        xmin -= buffer_size
        ymin -= buffer_size
        xmax += buffer_size
        ymax += buffer_size

        # Create a new EE Geometry with the buffered region
        bbox_4326 = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
    
    return epsg_code, bbox_4326

# Function to get relative elevation from ArcticDEM (Relative Elevation is More Context-Dependent)
def get_ArcticDEM_rel_el(kernel_size=100, offset=50, factor=300):
    dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")
    conv = dem.convolve(ee.Kernel.circle(kernel_size, 'meters'))
    diff = (dem.subtract(conv).add(ee.Image.constant(offset)).multiply(ee.Image.constant(factor)).toInt16())
    return diff

# Function to get slope from ArcticDEM
def get_ArcticDEM_slope():
    dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")
    slope = ee.Terrain.slope(dem)
    return slope

# Function to download image from GEE
def download_ee_image(image, region, scale, output_path):
    """Download Earth Engine image in EPSG:4326 and save as GeoTIFF."""
    
    url = image.getDownloadURL({
        'scale': scale,
        'region': region.getInfo(),
        'crs': "EPSG:4326",
        'format': 'GeoTIFF'
    })
    
    # Download file
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        temp_path = output_path.replace(".tif", "_4326.tif")  # Temporary file
        with open(temp_path, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"Downloaded: {temp_path}")
        return temp_path  # Return path of the downloaded file
    else:
        print(f"Failed to download {output_path}")
        return None

# Function to reproject with gdalwarp
def reproject_with_gdal(input_path, output_path, ref_image_path, epsg_code):
    """Reproject input image to match reference image using gdalwarp."""
    
    # Open the reference image with GDAL to get the CRS and geo-transform
    ref = gdal.Open(ref_image_path)
    geotransform = ref.GetGeoTransform()
    
    # Get bounding box from the reference image
    minX = geotransform[0]
    maxY = geotransform[3]
    pixSizeX = geotransform[1]
    pixSizeY = geotransform[5]
    
    # Calculate maxX and minY based on image size and pixel size
    xSize = ref.RasterXSize
    ySize = ref.RasterYSize
    maxX = minX + (xSize * pixSizeX)
    minY = maxY + (ySize * pixSizeY)
    
    # Build the gdalwarp command with the EPSG code
    bbStr = f' -te {minX} {minY} {maxX} {maxY} '
    resStr = f' -tr {pixSizeX} {pixSizeY} '
    projectionStr = f' -t_srs EPSG:{epsg_code} '  # Use the extracted EPSG code here
    warpOptions = ' -of GTiff ' + bbStr + resStr + projectionStr + ' -co COMPRESS=LZW -co BIGTIFF=YES '
    
    # Construct gdalwarp command
    warpCMD = f'gdalwarp {warpOptions}"{input_path}" "{output_path}"'
    
    # Log the command being executed
    #print(f"Running gdalwarp command: {warpCMD}")
    
    # Execute the command and capture output
    process = subprocess.Popen(warpCMD, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Log any errors or standard output from gdalwarp
    #if process.returncode != 0:
    #    print(f"gdalwarp failed with error: {stderr.decode()}")
    #else:
    #   print(f"gdalwarp executed successfully: {stdout.decode()}")
    
    #print(f"Reprojected to: {output_path}")

# Main function to process images
def download_dem_for_images(image_folder, output_folder, scale=10, buffer_size=0.05):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    failed_downloads = []

    for filename in os.listdir(image_folder):
        if filename.endswith(".tif"):
            filepath = os.path.join(image_folder, filename)
            name = os.path.splitext(filename)[0]

            # Extract metadata from the Sentinel-1 image
            epsg_code, region_4326 = get_image_metadata(filepath)

            # Get ArcticDEM elevation & slope
            elevation_layer = get_ArcticDEM_rel_el()
            slope_layer = get_ArcticDEM_slope()

            # Combine elevation and slope into a single image
            combined_image = ee.Image.cat(slope_layer, elevation_layer)

            # Temporary and final output paths
            temp_output = os.path.join(output_folder, f"{name}_temp.tif")
            final_output = os.path.join(output_folder, f"{name}.tif")

            # Download image in EPSG:4326
            downloaded_path = download_ee_image(combined_image, region_4326, scale, temp_output)

            if downloaded_path:
                # Reproject to original EPSG using the reference Sentinel-1 image
                reproject_with_gdal(downloaded_path, final_output, filepath, epsg_code)

                # Remove temporary file
                os.remove(downloaded_path)
            else:
                failed_downloads.append(filename)
                
    # Summary of failed downloads
    if failed_downloads:
        print("\nSummary: Failed to download the following files:")
        for failed_file in failed_downloads:
            print(f"- {failed_file}")
    else:
        print("\nAll files downloaded and reprojected successfully!")

In [4]:
# Example usage
image_folder = r"C:\Users\smola\Documents\MASTER\TRY_Dataset\test\S1"
output_folder = r"C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM"
download_dem_for_images(image_folder, output_folder)

Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\0A02_w_3826_temp_4326.tif


c:\Users\smola\miniconda3\envs\myunet\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\0A02_w_4184_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\1D90_w_1347_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\1D90_w_2362_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20160821T181425_w_188_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20160821T181425_w_83_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20180706T222019_w_4_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20180706T222019_w_8_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20180712T181352_w_277_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20190619T222024_w_11_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\test\DEM\20190627T020825_w_18_temp_4326.tif
Downloaded: C:\Users\smola\

In [5]:
# Example usage
image_folder = r"C:\Users\smola\Documents\MASTER\TRY_Dataset\train\S1"
output_folder = r"C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM"
download_dem_for_images(image_folder, output_folder)

Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_11561_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_11562_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_11830_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_3914_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_3916_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_4004_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_4005_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_4337_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_5351_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\0A02_w_5441_temp_4326.tif
Downloaded: C:\Users\smola\Documents\MASTER\TRY_Dataset\train\DEM\1D90_w_1565

In [ ]:
# Example usage
image_folder = r"C:\Users\smola\Documents\MASTER\TRY_Dataset\val\S1"
output_folder = r"C:\Users\smola\Documents\MASTER\TRY_Dataset\val\DEM"
download_dem_for_images(image_folder, output_folder)